In [ ]:
import shutil

import xlwings as xw
import pandas as pd
from collections import defaultdict
import os
from upath import UPath
from loguru import logger
import sys
from datetime import datetime

from runner import *

logger.remove()
logger.add(sys.stderr, backtrace=False)

def run_and_load_case(*, wb: xw.Book):
    base_folder = save_multistage_case(wb=wb)
    !julia --project=. Run.jl $base_folder
    load_case_results(wb=wb, base_folder=base_folder)

wb = xw.Book(
    '/Users/roderick/Library/CloudStorage/OneDrive-SharedLibraries-ResilientTransition/5.001 Kentucky Resource Council - Documents/Data/Kentucky Load Resource Model.xlsb'
)

# Dispatch

In [ ]:
import plotly.graph_objects as go
import plotly.io as pio

axes = dict(
    showgrid=False,
    linecolor="rgb(120, 120, 120)",
    linewidth=1,
    showline=True,
    ticks="outside",
    tickcolor="rgb(120, 120, 120)",
    mirror=True,
)

pio.templates["e3"] = go.layout.Template(
    layout=go.Layout(
        font=dict(family="CommitMono", size=11, color="rgb(120, 120, 120)"),
        title=dict(
            font=dict(
                # size=32,
                color="rgb(3, 78, 110)",
            ),
            x=0.06,
            y=0.92,
            xanchor="left",
            yanchor="bottom",
        ),
        xaxis=axes,
        yaxis=axes,
        margin=dict(t=60, b=100, r=60, l=60),
    )
)

pio.templates["2.5x7.5"] = go.layout.Template(
    layout=go.Layout(
        height=2.5 * 144,
        width=7.5 * 144,
    )
)

pio.templates.default = "e3+2.5x7.5"

In [ ]:
type_map = wb.sheets["GenX Resources"].range("TypeMap").options(pd.DataFrame, index=0).value.dropna().set_index("resource").squeeze(axis=1).to_dict()

In [ ]:
color_map = {
"Coal": "black",
"Gas Other": "#434343",
"Gas CCGT": "#7F7F7F",
"Gas CT": "#BFBFBF",
"Nuclear": "#FF8AD8",
"Biogas": "#7F4910",
"Hydro": "#0070C0",
"Wind": "#76F2FF",
"Solar": "#FFC000",
"4-hr Battery": "#7030A0",
"8-hr Battery": "#7030A0",
"100-hr Battery": "#002060",
"Capacity Purchase": "#C00000",
}

pattern_shape_map = {
"Coal": "",
"Gas Other": "",
"Gas CCGT": "",
"Gas CT": "",
"Nuclear": "",
"Biogas": "",
"Hydro": "",
"Wind": "",
"Solar": "",
"4-hr Battery": "",
"8-hr Battery": "",
"100-hr Battery": "/",
"Capacity Purchase": "/",
}

In [ ]:
base_folder = UPath("/Users/roderick/PycharmProjects/resilient-transition/GenX.jl/cases/2025-06-05/KRC-6Years-24Days-NoPlan-Retire-BBB-NoGasAfter2040-Wind1GW/pcm/results/results_p1")

df = pd.read_csv(base_folder / "power.csv").iloc[2:, 1:-1]
df = df.reset_index(drop=True)
df.index = pd.Timestamp("1/1/2007") + pd.to_timedelta(df.index, unit="h")
df = df.T.groupby(type_map).sum().T
df = df[wb.sheets["List"].range("Clusters").options(list).value]
df = df[[col for col in df.columns if df[col].sum()>0]]

load = -pd.read_csv(base_folder / "power_balance.csv", skiprows=[1, 2], index_col=0).iloc[:, -1].squeeze()
load.index = df.index

import plotly.graph_objects as go

# Hourly
fig = go.Figure(
    data=[
        go.Scatter(x=df.index, y=df[col], fill="tonexty", fillcolor=color_map[col], stackgroup=1, name=col)
        for col in df
    ]
)
fig.update_traces(line_width=0)
fig.add_trace(go.Scatter(x=df.index, y=load, mode="lines", name=load.name, stackgroup=None, line_color="black"))
fig.show()

## Month-hour
month_hour = df.groupby(by=[df.index.month, df.index.hour]).mean()
month_hour = month_hour.reset_index(names=["month", "hour"])

# blanks = pd.DataFrame(index=[(24 * n) + 0.5 for n in range(1, 12)])
# blanks["month"] = blanks.index // 24
# blanks["hour"] = 24.5

# month_hour = pd.concat([month_hour, blanks], axis=0).sort_index()

mh_fig = go.Figure(
    data=[
        go.Scatter(x=[month_hour["month"], month_hour["hour"]], y=month_hour[col], fill="tonexty", fillcolor=color_map[col], stackgroup=1, name=col, connectgaps=False)
        for col in month_hour if col not in ["month", "hour"]
    ]
)
for i in range(1, 12):
    mh_fig.add_vrect(x0=i*24 - 1, x1=i*24, line_width=0, fillcolor="white")

mh_fig.update_traces(line_width=0)
mh_load = load.groupby(by=[load.index.month, load.index.hour]).mean()
mh_load = mh_load.to_frame().reset_index(names=["month", "hour"])
mh_fig.add_trace(go.Scatter(x=[mh_load["month"], mh_load["hour"]], y=mh_load["Demand"], name="Demand", stackgroup=None, mode="lines", line_color="black"))

mh_fig.update_layout(title="<b>IRA: No New Gas + Limited Wind</b><br>2050 Month-Hour Average Dispatch")
# mh_fig.update_xaxes(tickson="labels")
mh_fig.show()